In [2]:
pip install findspark

  Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as ps
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [2]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("LinearRegWithSpark")\
        .getOrCreate()

In [3]:
dataset = spark.read.csv(r"C:\Users\Harshit\Downloads\boston.csv",header=True)

In [8]:
dataset.show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|  18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|   0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|   0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|   0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|   0|2.

In [5]:
dataset.withColumnRenamed("MV","Price").show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|      Price|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|  18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|   0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|   0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|   0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|   0|2.

In [9]:
dataset.select("MV").show()

+-----------+
|         MV|
+-----------+
|         24|
|21.60000038|
|34.70000076|
|33.40000153|
|36.20000076|
|28.70000076|
|22.89999962|
|27.10000038|
|       16.5|
|18.89999962|
|         15|
|18.89999962|
|21.70000076|
|20.39999962|
|18.20000076|
|19.89999962|
|23.10000038|
|       17.5|
|20.20000076|
|18.20000076|
+-----------+
only showing top 20 rows



In [10]:
dataset.printSchema()

root
 |-- CRIM: string (nullable = true)
 |-- ZN: string (nullable = true)
 |-- INDUS: string (nullable = true)
 |-- CHAS: string (nullable = true)
 |-- NOX: string (nullable = true)
 |-- RM: string (nullable = true)
 |-- AGE: string (nullable = true)
 |-- DIS: string (nullable = true)
 |-- RAD: string (nullable = true)
 |-- TAX: string (nullable = true)
 |-- PT: string (nullable = true)
 |-- B: string (nullable = true)
 |-- LSTAT: string (nullable = true)
 |-- MV: string (nullable = true)



In [11]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [12]:
new_data.printSchema()

root
 |-- CRIM: float (nullable = true)
 |-- ZN: float (nullable = true)
 |-- INDUS: float (nullable = true)
 |-- CHAS: float (nullable = true)
 |-- NOX: float (nullable = true)
 |-- RM: float (nullable = true)
 |-- AGE: float (nullable = true)
 |-- DIS: float (nullable = true)
 |-- RAD: float (nullable = true)
 |-- TAX: float (nullable = true)
 |-- PT: float (nullable = true)
 |-- B: float (nullable = true)
 |-- LSTAT: float (nullable = true)
 |-- MV: float (nullable = true)



In [13]:
from pyspark.sql.functions import col, count, isnan, when

In [14]:
#let's check for missing values

In [15]:
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+----+---+-----+----+---+---+---+---+---+---+---+---+-----+---+
|CRIM| ZN|INDUS|CHAS|NOX| RM|AGE|DIS|RAD|TAX| PT|  B|LSTAT| MV|
+----+---+-----+----+---+---+---+---+---+---+---+---+-----+---+
|   0|  0|    0|   0|  0|  0|  0|  0|  0|  0|  0|  0|    0|  0|
+----+---+-----+----+---+---+---+---+---+---+---+---+-----+---+



In [16]:
features = new_data.drop('MV')

In [17]:
#let's assemble our features together using vectorAssembler
assembler = VectorAssembler(
    inputCols=features.columns,
    outputCol="features")

In [18]:
output = assembler.transform(new_data)

In [19]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[0.00632000016048...|
|[0.02731000073254...|
|[0.02728999964892...|
|[0.03237000107765...|
|[0.06904999911785...|
|[0.02985000051558...|
|[0.08828999847173...|
|[0.14454999566078...|
|[0.21123999357223...|
|[0.17003999650478...|
|[0.22488999366760...|
|[0.11747000366449...|
|[0.09378000348806...|
|[0.62976002693176...|
|[0.63796001672744...|
|[0.62739002704620...|
|[1.05393004417419...|
|[0.78420001268386...|
|[0.80270999670028...|
|[0.72579997777938...|
+--------------------+
only showing top 20 rows



In [20]:
output= output.select("features", "MV")

In [21]:
train_df,test_df = output.randomSplit([0.7, 0.3])

In [22]:
train_df.show()
test_df.show()

+--------------------+----+
|            features|  MV|
+--------------------+----+
|[0.00632000016048...|24.0|
|[0.00906000006943...|32.2|
|[0.01095999963581...|22.0|
|[0.01300999987870...|32.7|
|[0.01310999970883...|35.4|
|[0.01360000018030...|18.9|
|[0.01381000038236...|50.0|
|[0.01432000007480...|31.6|
|[0.01439000014215...|29.1|
|[0.01501000020653...|24.5|
|[0.01501000020653...|50.0|
|[0.01537999976426...|44.0|
|[0.01709000021219...|30.1|
|[0.01869999989867...|23.1|
|[0.01950999908149...|33.0|
|[0.02009000070393...|50.0|
|[0.02177000045776...|42.3|
|[0.02187000028789...|31.1|
|[0.02497999928891...|16.5|
|[0.02542999945580...|23.9|
+--------------------+----+
only showing top 20 rows

+--------------------+----+
|            features|  MV|
+--------------------+----+
|[0.01778000034391...|32.9|
|[0.01964999921619...|20.1|
|[0.02054999954998...|24.7|
|[0.03048999980092...|31.2|
|[0.03444999828934...|24.1|
|[0.03502000123262...|28.5|
|[0.03550999984145...|22.9|
|[0.03658999875187...|

In [111]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='MV')
linear_model = lin_reg.fit(train_df)

In [112]:
linear_model.coefficients

DenseVector([-0.1197, 0.056, -0.0428, 2.509, -15.4722, 3.2663, -0.0044, -1.7609, 0.3636, -0.0146, -0.8708, 0.0108, -0.5595])

In [113]:
print("Coefficients: " + str(linear_model.coefficients))
print("Intercept: " + str(linear_model.intercept))

Coefficients: [-0.11970588021808723,0.055982666172624575,-0.0428180435109268,2.5090340854479796,-15.472234018178732,3.266330242614088,-0.0044154088217970575,-1.760890198008908,0.3635974012020112,-0.014593039754557204,-0.8707516463132695,0.010836540914100261,-0.5594746315647217]
Intercept: 39.49154298705353


In [114]:
linear_model.summary.residuals

DataFrame[residuals: double]

In [115]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("r2: %f" % trainSummary.r2)

RMSE: 4.580971
r2: 0.750854


In [116]:
# Testing the predictions
pred_results=linear_model.evaluate(test_df)

In [117]:
pred_results.predictions.show(40)

+--------------------+----+------------------+
|            features|  MV|        prediction|
+--------------------+----+------------------+
|[0.01310999970883...|35.4|  31.0409829105355|
|[0.01360000018030...|18.9|15.201113227456204|
|[0.01432000007480...|31.6|  33.0794828036544|
|[0.01537999976426...|44.0|37.442318955262465|
|[0.01778000034391...|32.9|30.373043789166417|
|[0.01950999908149...|33.0|22.172902418288583|
|[0.02187000028789...|31.1|32.094691467287184|
|[0.02497999928891...|16.5|21.202632001139403|
|[0.02762999944388...|30.8| 32.22187501896185|
|[0.02899000048637...|26.6| 21.31752133266605|
|[0.03040999919176...|18.5|19.732221613996465|
|[0.03112999908626...|17.5|15.651258121270576|
|[0.03426999971270...|19.5|20.829414199884937|
|[0.03466000035405...|19.4|23.071295845796353|
|[0.03502000123262...|28.5|34.270716528885785|
|[0.03536999970674...|22.0|28.805455672721724|
|[0.03548000007867...|20.9|21.211329442763837|
|[0.03550999984145...|22.9|25.355806113383267|
|[0.035780001

In [118]:
# prediction

predictions = linear_model.transform(test_df)
predictions.select("prediction","MV","features").show()

+------------------+----+--------------------+
|        prediction|  MV|            features|
+------------------+----+--------------------+
|  31.0409829105355|35.4|[0.01310999970883...|
|15.201113227456204|18.9|[0.01360000018030...|
|  33.0794828036544|31.6|[0.01432000007480...|
|37.442318955262465|44.0|[0.01537999976426...|
|30.373043789166417|32.9|[0.01778000034391...|
|22.172902418288583|33.0|[0.01950999908149...|
|32.094691467287184|31.1|[0.02187000028789...|
|21.202632001139403|16.5|[0.02497999928891...|
| 32.22187501896185|30.8|[0.02762999944388...|
| 21.31752133266605|26.6|[0.02899000048637...|
|19.732221613996465|18.5|[0.03040999919176...|
|15.651258121270576|17.5|[0.03112999908626...|
|20.829414199884937|19.5|[0.03426999971270...|
|23.071295845796353|19.4|[0.03466000035405...|
|34.270716528885785|28.5|[0.03502000123262...|
|28.805455672721724|22.0|[0.03536999970674...|
|21.211329442763837|20.9|[0.03548000007867...|
|25.355806113383267|22.9|[0.03550999984145...|
|38.387902312

In [119]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R Squared (R2) on test data = %g" % pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.708144


In [120]:
spark.stop()